# Import necessary modules and libraries

In [5]:
import PyQt5
%config InlineBackend.figure_format = 'retina'
%matplotlib qt5

#python packages
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pac
import warnings

import seaborn as sns
sns.set_style('white')

import imp
import shape
import utils
from load_features import load_WaveformShape_features, Bandpower_features, mean_and_peak_freqs, statistics, fractal_dimensions, entropies
from val_metrics import *

imp.reload(utils)
imp.reload(shape)

<module 'shape' from 'C:\\Users\\USER\\Documents\\Yachay_Tech\\Thesis_Project\\ParkinsonsDetection\\python_scripts\\shape.py'>

# Load data and meta-data

In [6]:
"""Which comparison to make:
    1. Off-med vs Controls
    2. On-med vs Controls
    3. Off-med vs On-med
    """
comparison = 2
dataset = 'UCSD' #UCSD or UNM

In [7]:
"""Which 'comparison' to make:
    1. Off-med vs Controls
    2. On-med vs Controls
    3. Off-med vs On-med
    """
all_chan = True; EO = False
bands = [[0.5,4], [4,8], [8,12], [16,32], [32,64]] #Delta, Theta, Alpha, Beta, Gamma

In [8]:
Fs, t, S, Sc, Smed, flo, fhi = utils.loadmeta() 
eeg,rejects = utils.loadPD(EO, all_chan, dataset) # EO means Eyes Opened

# Ploting a signal

In [10]:
# Plot a signal
sns.set(font_scale=1.2)
data = eeg['on'][15]
time = np.arange(data.size) / Fs

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plt.plot(time, data, lw=1.5, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Voltage')
plt.xlim([time.min(), time.max()])
plt.title('EEG Signal for PD patients on-medication')
sns.despine()

# plt.plot(eeg['off'][6,0:10],label='control')
# plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))

# Waveform Shape Features

In [ ]:
"""
This cell saves features from waveform shape and PAC in .pkl files -- Just run once, then commented--
"""

# widthS = 3

# pks,trs,ShR,PTR,StR,RDR = utils.measure_shape(eeg, rejects, widthS=widthS)

"""
Algorithms for estimating phase-amplitude coupling
"""
# pac = utils.measure_pac(eeg,rejects,flo,fhi,Fs=Fs)

# import pickle
# shape_features = [ShR, PTR, StR, RDR, pac]
# shape_featuresStr = ["ShR", "PTR", "StR", "RDR", "pac"]
# for i in range(len(shape_features)):
#     f = open(shape_featuresStr[i] + ".pkl","wb")
#     pickle.dump(shape_features[i],f)
#     f.close()

In [ ]:
"""This function calculates the shape measures calculated for analysis
    of the PD data set

    1. Peak and trough times(pks,trs)
    2. Peak and trough sharpness(pksharp,trsharp)
    3. Rise and decay steepnes(risteep,desteep)
    3. Sharpness ratio(ShR)
    4. Steepness ratio(StR)
    5. Peak-to-trough ratio(PTR)
    6. Rise-to-decay ratio(RDR)
    """
# ShR, PTR, StR, RDR, pac = load_WaveformShape_features(EO, dataset)

In [5]:
"""This function calculates the shape measures calculated for analysis
    of the PD data set

    1. Peak and trough times(pks,trs)
    2. Peak and trough sharpness(pksharp,trsharp)
    3. Rise and decay steepnes(risteep,desteep)
    3. Sharpness ratio(ShR)
    4. Steepness ratio(StR)
    5. Peak-to-trough ratio(PTR)
    6. Rise-to-decay ratio(RDR)
    """
widthS = 3 #To calculate Waveform Shape features

pks,trs,ShR,PTR,StR,RDR = utils.measure_shape(eeg, rejects, widthS=widthS)
"""
Algorithms for estimating phase-amplitude coupling
"""
pac = utils.measure_pac(eeg,rejects,flo,fhi,Fs=Fs)

# Calculate Spectral Features

In [6]:
"""
Absolute and Relative BandPower features, from all five bands: Delta, Theta, Alpha, Beta, Gamma
"""
abs_powerOff = Bandpower_features(eeg['off'], Fs, bands, S, False, 'welch')
abs_powerOn = Bandpower_features(eeg['on'], Fs, bands, Smed, False, 'welch')
abs_powerCtl = Bandpower_features(eeg['C'], Fs, bands, Sc, False, 'welch')

rel_powerOff = Bandpower_features(eeg['off'], Fs, bands, S, True, 'welch')
rel_powerOn = Bandpower_features(eeg['on'], Fs, bands, Smed, True, 'welch')
rel_powerCtl = Bandpower_features(eeg['C'], Fs, bands, Sc, True, 'welch')

In [7]:
"""
Mean and Peak Frequency from the spectrum
"""
meanFreqsOff = mean_and_peak_freqs(eeg['off'], Fs, S)[0] 
meanFreqsOn = mean_and_peak_freqs(eeg['on'], Fs, Smed)[0]
meanFreqsCtl = mean_and_peak_freqs(eeg['C'], Fs, Sc)[0]

peakFreqsOff = mean_and_peak_freqs(eeg['off'], Fs, S)[1]
peakFreqsOn = mean_and_peak_freqs(eeg['on'], Fs, Smed)[1]
peakFreqsCtl = mean_and_peak_freqs(eeg['C'], Fs, Sc)[1]

# Statistical Features

In [8]:
"""This cell calculates statistical measures extracted from EEG for analysis
    of the PD data set

    1. Mean
    2. Standard Deviation
    3. Skewness
    4. Kurtosis
    5. Maximum
    6. Minimum
    7. 5th percentile value
    8. 25th percentile value
    9. 75th percentile value
    10. 95th percentile value
    11. Median
    12. Variance
    13. Root Mean Square value
    """
statsOff = statistics(eeg['off'], S).get()
statsOn = statistics(eeg['on'], Smed).get()
statsCtl = statistics(eeg['C'], Sc).get()

# Non-linear analysis

In [9]:
fractalOff = fractal_dimensions(eeg['off'], S)
fractalOn = fractal_dimensions(eeg['on'], Smed)
fractalCtl = fractal_dimensions(eeg['C'], Sc)

In [10]:
entOff = entropies(eeg['off'], S, Fs)
entOn = entropies(eeg['on'], Smed, Fs)
entCtl = entropies(eeg['C'], Sc, Fs)

# Load dataset with all features

In [81]:
# create features of class I
f1_B    = np.reshape(pac['off'],(S,1))
f2_B    = np.reshape(ShR['off'],(S,1))
f3_B    = np.reshape(StR['off'],(S,1))
f4_B    = np.reshape(PTR['off'],(S,1))
f5_B    = np.reshape(RDR['off'],(S,1))
cl_B    = np.ones((S,1)) # 1

In [82]:
# create features of class II
f1_C    = np.reshape(pac['on'],(Smed,1))
f2_C    = np.reshape(ShR['on'],(Smed,1))
f3_C    = np.reshape(StR['on'],(Smed,1))
f4_C    = np.reshape(PTR['on'],(Smed,1))
f5_C    = np.reshape(RDR['on'],(Smed,1))
if comparison == 1 or comparison == 3:
    cl_C    = np.zeros((Smed,1)) # transition means 0 #Original line
elif comparison == 2:
    cl_C    = np.ones((Smed,1))

In [83]:
# create features of class III
f1_E    = np.reshape(pac['C'],(Sc,1))
f2_E    = np.reshape(ShR['C'],(Sc,1))
f3_E    = np.reshape(StR['C'],(Sc,1))
f4_E    = np.reshape(PTR['C'],(Sc,1))
f5_E    = np.reshape(RDR['C'],(Sc,1))
cl_E    = np.negative(np.ones((Sc,1))) # -1

In [84]:
MftB = np.concatenate([f1_B,f2_B,f3_B, f4_B, f5_B, rel_powerOff, abs_powerOff, meanFreqsOff, peakFreqsOff, statsOff, fractalOff, entOff, cl_B],axis=1)
MftC = np.concatenate([f1_C,f2_C,f3_C, f4_C, f5_C, rel_powerOn,  abs_powerOn,  meanFreqsOn, peakFreqsOn,   statsOn,  fractalOn, entOn,  cl_C],axis=1)
MftE = np.concatenate([f1_E,f2_E,f3_E, f4_E, f5_E, rel_powerCtl, abs_powerCtl, meanFreqsCtl, peakFreqsCtl, statsCtl, fractalCtl, entCtl, cl_E],axis=1)

In [85]:
features = ['PAC','ShR','StR', 'PtT', 'RtF', 'rel_delta',
            'rel_theta','rel_alpha','rel_beta',
            'rel_gamma','abs_delta','abs_theta',
            'abs_alpha','abs_beta','abs_gamma','meanFreq','peakFreq',
            'mean','std','skewness', 'kurtosis', 'maximum', 'minimum',
            '5th perc','25th perc','75th perc','95th perc','median','variance','RMS',
           'detrended_fluctuation', 'higuchi_fd', 'katz_fd', 'petrosian_fd',
           'perm_entropy', 'svd_entropy']

In [86]:
FCM_B = pd.DataFrame(MftB,columns= features + ['class'])
FCM_C = pd.DataFrame(MftC,columns= features + ['class'])
FCM_E = pd.DataFrame(MftE,columns= features + ['class'])

In [87]:
#Classification between patients on-medication and patients off-medication   

if comparison == 3:
    TotalDataset = pd.concat([FCM_B,FCM_C],ignore_index=True)
    visDat = TotalDataset.copy(deep=True)
    visDat['class'] = visDat['class'].map({1:'off_med',0:'on_med'})

#Classification between patients on-medication and healthy control subjects        

elif comparison == 2:
    TotalDataset = pd.concat([FCM_C,FCM_E],ignore_index=True)
    visDat = TotalDataset.copy(deep=True)
    # visDat['class'] = visDat['class'].map({-1:'control',0:'on_med'}) #Original line
    visDat['class'] = visDat['class'].map({-1:'control',1:'on_med'})

#Classification between patients off-medication and healthy control subjects        

elif comparison == 1:
    TotalDataset = pd.concat([FCM_E,FCM_B],ignore_index=True)
    visDat = TotalDataset.copy(deep=True)
    visDat['class'] = visDat['class'].map({-1:'control',1:'off_med'})

In [88]:
visDat.head(3240)

,PAC,ShR,StR,PtT,RtF,rel_delta,rel_theta,rel_alpha,rel_beta,rel_gamma,...,median,variance,RMS,detrended_fluctuation,higuchi_fd,katz_fd,petrosian_fd,perm_entropy,svd_entropy,class
0,0.004507,0.015101,0.007890,0.965826,1.018334,0.160536,0.188544,0.215391,0.290536,0.105444,...,-0.147196,44.335599,5.200195,0.694626,1.329258,3.614550,1.010760,2.233689,0.900570,off_med
1,0.048979,0.081868,0.003167,0.828194,1.007320,0.093071,0.121841,0.080416,0.206895,0.325862,...,-0.184526,137.111610,9.178630,0.660148,1.699939,4.506174,1.011811,2.305504,1.285569,off_med
2,0.009106,0.010461,0.048192,1.024380,1.117357,0.315638,0.177761,0.159741,0.207027,0.225231,...,-0.083672,13.698079,2.873184,0.794236,1.627005,3.901378,1.012982,2.362892,1.136908,off_med
3,0.033689,0.045132,0.026650,0.901297,1.063287,0.125193,0.065370,0.082224,0.168007,0.265845,...,0.084805,64.883761,6.245067,0.665163,1.792458,4.740969,1.013654,2.422854,1.385232,off_med
4,0.015581,0.025718,0.008366,0.942502,1.019449,0.167847,0.092750,0.104343,0.220804,0.311636,...,-0.313019,29.723154,4.262029,0.756551,1.664885,4.312527,1.012126,2.317893,1.228517,off_med
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,0.054289,0.035911,0.094564,1.086204,0.804333,0.137295,0.217630,0.206912,0.179459,0.206942,...,-0.023963,43.464109,5.143440,0.640378,1.611932,4.277311,1.012396,2.334022,1.180773,on_med
933,0.026948,0.052830,0.116053,1.129353,0.765503,0.257445,0.165237,0.115304,0.185560,0.227640,...,0.028889,24.561608,3.877497,0.716123,1.720568,4.457114,1.013577,2.398445,1.295304,on_med
934,0.065171,0.023796,0.121652,1.056320,0.755698,0.142134,0.245133,0.243544,0.176670,0.192061,...,0.173929,84.545968,7.233617,0.666495,1.540297,3.980859,1.012198,2.324535,1.138620,on_med
935,0.034843,0.032679,0.098393,1.078151,0.797273,0.192164,0.089684,0.069790,0.130481,0.292350,...,0.029313,130.151138,7.674405,0.479596,1.724914,3.362345,1.011705,2.315616,1.371452,on_med


In [ ]:
interval = np.arange(0,1)
off = FCM_B.iloc[:,interval].mean(axis=0)
on = FCM_C.iloc[:,interval].mean(axis=0)
control = FCM_E.iloc[:,interval].mean(axis=0)

df = pd.DataFrame({'PD Off-medication': off,
                   'PD On-medication': on,
                   'Healthy': control}, index=np.asarray(features)[interval])
ax = df.plot.bar(rot=0)

# Selecting the set of features

In [89]:
X = TotalDataset[features]#.iloc[:,5:10]
y = TotalDataset[['class']]
X = np.asarray(X)
y = np.asarray(y)

# PCA

In [ ]:
# import pandas as pd
# from sklearn import datasets
# from sklearn.decomposition import PCA

# # load dataset
# df = pd.DataFrame(X, columns=features)

# # Standarize Data
# from sklearn import preprocessing
# scaler = preprocessing.StandardScaler().fit(df)
# data_scaled = pd.DataFrame(scaler.transform(df),columns = df.columns) 

# # PCA
# pca = PCA(.95)
# # pca.fit_transform(data_scaled)
# X = pca.fit_transform(data_scaled)

In [ ]:
# # get component loadings (correlation coefficient between original variables and the component) 
# # the squared loadings within the PCs always sums to 1
# loadings = pca.components_
# num_pc = pca.n_features_
# pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
# loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
# loadings_df['variable'] = df.columns.values
# loadings_df = loadings_df.set_index('variable')

# # positive and negative values in component loadings reflects the positive and negative correlation of the variables
# # with then PCs. 

# # get correlation matrix plot for loadings
# import seaborn as sns
# import matplotlib.pyplot as plt
# ax = sns.heatmap(loadings_df, annot=True, cmap='coolwarm')
# plt.show()

# ANOVA Feature selection, SelectKBest

In [90]:
#Scale Dataset
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

# ANOVA feature selection for numeric input and categorical output
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, SelectFpr
from sklearn.feature_selection import f_classif, f_regression
# define feature selection
fs = SelectKBest(score_func=f_classif, k=20)
# fs = SelectKBest()
# fs = SelectFpr(score_func=f_classif, alpha=0.01)
# apply feature selection
X = fs.fit_transform(X, np.ravel(y))

print("The shape of the new dataset is = " + str(X.shape))

The shape of the new dataset is = (937, 20)


In [91]:
X_indices = np.arange(X.shape[-1])
selector = SelectKBest(f_classif, k=4)
selector.fit(X, np.ravel(y))
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
plt.bar(X_indices , scores, width=.2,
        label=r'Univariate score ($-Log(p_{value})$)')


plt.title("Comparing feature selection")
plt.xlabel('Feature number')
# plt.yticks(())
plt.axis('tight')
plt.legend(loc='upper right')
plt.show()

In [22]:
# df = pd.DataFrame(fs.pvalues_,columns= ['pvalues'],index = features)
# df.sort_values(by=['pvalues'], ascending=True)

Load Sklearn libraries

In [92]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import classification_report

# Cross-validation scores

In [93]:
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
# prepare configuration for cross validation test harness
seed = 151461
folds = model_selection.ShuffleSplit(n_splits=10, test_size=0.10, random_state=seed)

# Gaussian Process

In [94]:
clf = GaussianProcessClassifier()
accuraccy_Gaussian, f1_Gaussian = cross_vald(clf, folds, X, y)

Accuracy: 0.717 (+/- 0.053)
F1-Score: 0.719 (+/- 0.052)
Precision: 0.702 (+/- 0.083)
Recall: 0.742 (+/- 0.106)


In [ ]:
roc_curve(clf, folds, X, y)

# Gradient Boosting Classifier

In [ ]:
clf = GradientBoostingClassifier(learning_rate = 0.25, max_depth =2, n_estimators=100)
accuraccy_GradBoost, f1_GradBoost = cross_vald(clf, folds, X, y)

In [ ]:
roc_curve(clf, folds, X, y)

# SVM

In [95]:
clf = SVC(C=80, kernel = 'rbf', degree = 5, gamma = 'auto')
accuraccy_SVC, f1_SVC = cross_vald(clf, folds, X, y)

Accuracy: 0.720 (+/- 0.093)
F1-Score: 0.724 (+/- 0.080)
Precision: 0.703 (+/- 0.066)
Recall: 0.748 (+/- 0.123)


In [ ]:
roc_curve(clf, folds, X, y)

# KNN

In [96]:
clf = KNeighborsClassifier(n_neighbors = 4, p=2, weights='distance')
accuraccy_kNN, f1_kNN = cross_vald(clf, folds, X, y)

Accuracy: 0.726 (+/- 0.091)
F1-Score: 0.726 (+/- 0.102)
Precision: 0.707 (+/- 0.076)
Recall: 0.747 (+/- 0.145)


In [ ]:
roc_curve(clf, folds, X, y)

# Random Forest

In [97]:
clf = RandomForestClassifier(criterion = 'entropy', max_depth = None,
                             max_features = 'log2', min_samples_split = 2, 
                             n_estimators = 350)
accuraccy_RF, f1_RF = cross_vald(clf, folds, X, y)

Accuracy: 0.753 (+/- 0.074)
F1-Score: 0.757 (+/- 0.070)
Precision: 0.723 (+/- 0.077)
Recall: 0.804 (+/- 0.113)


In [ ]:
roc_curve(clf, folds, X, y)

# MLP

In [98]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [99]:
clf = MLPClassifier(activation = 'relu', alpha = 0.0001, 
                    hidden_layer_sizes = (50,50,50), 
                    learning_rate = 'constant', solver = 'lbfgs')
accuraccy_MLP, f1_MLP = cross_vald(clf, folds, X, y)

Accuracy: 0.738 (+/- 0.086)
F1-Score: 0.752 (+/- 0.075)
Precision: 0.746 (+/- 0.102)
Recall: 0.740 (+/- 0.082)


In [ ]:
roc_curve(clf, folds, X, y)

# Accuraccy and F1-score boxplots

In [100]:
acc = np.concatenate([np.reshape(accuraccy_Gaussian,(folds.get_n_splits(),1)),
                      np.reshape(accuraccy_SVC,(folds.get_n_splits(),1)), 
                      np.reshape(accuraccy_kNN,(folds.get_n_splits(),1)), 
                      np.reshape(accuraccy_RF,(folds.get_n_splits(),1)), 
                      np.reshape(accuraccy_MLP,(folds.get_n_splits(),1))],axis=1)
f1 = np.concatenate([np.reshape(f1_Gaussian,(folds.get_n_splits(),1)), 
                     np.reshape(f1_SVC,(folds.get_n_splits(),1)),
                     np.reshape(f1_kNN,(folds.get_n_splits(),1)), 
                     np.reshape(f1_RF,(folds.get_n_splits(),1)), 
                     np.reshape(f1_MLP,(folds.get_n_splits(),1))],axis=1)

In [101]:
accdf = pd.DataFrame(acc,columns=['GausProc', 'SVC', 'KNN', 'RandFor', 'MLP'])
f1df = pd.DataFrame(f1,columns=['GausProc', 'SVC', 'KNN', 'RandFor', 'MLP'])

In [102]:
boxplots(EO, comparison, accdf, f1df)